# **Glimpse of the Store Data**

In [1]:
# Data Processing
import numpy as np
import pandas as pd
from datetime import datetime

#Data Visualizations
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px 
from scipy import stats

In [2]:
import os, sys
sys.path.append('../scripts')

In [3]:
store_data = pd.read_csv('../data/raw/store.csv',
                        low_memory=False,)
train_df = pd.read_csv('../data/preprocessed/train_df.csv')

In [4]:
store_data.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [5]:
#checking the nulls in the store data
store_data.isnull().sum()/store_data.shape[0] * 100 

Store                         0.000000
StoreType                     0.000000
Assortment                    0.000000
CompetitionDistance           0.269058
CompetitionOpenSinceMonth    31.748879
CompetitionOpenSinceYear     31.748879
Promo2                        0.000000
Promo2SinceWeek              48.789238
Promo2SinceYear              48.789238
PromoInterval                48.789238
dtype: float64

Insights:
- The Promo2SinceWeek,Promo2SinceYear, and PromoInterval variables has 48% NULL values cause there are no continous promotion for those stores.
- Instead for CompetitionOpensinceMonth and CompetitionOpensinceYear ,it's basically missing data ( ~31.7) , this means that we have the nearest
distance of the competitor but miss the date information on when did he actually opened next to the this store chain.

CompetitionDistance - distance in meters to the nearest competitor store

In [6]:
# rows with missing values for competition distance
store_data[pd.isnull(store_data.CompetitionDistance)]

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
290,291,d,a,NaN,NaN,NaN,0,NaN,NaN,NaN
621,622,a,c,NaN,NaN,NaN,0,NaN,NaN,NaN
878,879,d,a,NaN,NaN,NaN,1,5.0,2013.0,"Feb,May,Aug,Nov"


- only 3 rows with null values, we need to impute or remove those rows.

In [7]:
store_data.describe()

,Store,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear
count,1115.00000,1112.000000,761.000000,761.000000,1115.000000,571.000000,571.000000
mean,558.00000,5404.901079,7.224704,2008.668857,0.512108,23.595447,2011.763573
std,322.01708,7663.174720,3.212348,6.195983,0.500078,14.141984,1.674935
min,1.00000,20.000000,1.000000,1900.000000,0.000000,1.000000,2009.000000
25%,279.50000,717.500000,4.000000,2006.000000,0.000000,13.000000,2011.000000
50%,558.00000,2325.000000,8.000000,2010.000000,1.000000,22.000000,2012.000000
75%,836.50000,6882.500000,10.000000,2013.000000,1.000000,37.000000,2013.000000
max,1115.00000,75860.000000,12.000000,2015.000000,1.000000,50.000000,2015.000000


- The distribution of CompetitionDistance variable is highly skewed with a noticeable gap between the mean and median. This is due to a high degree of d dispersion in the data, as indicated by a standard deviation of 7663, which is higher than both the mean and median.
- So, It is more advisable to use the median rather than the mean to fill in the three missing store values, as the mean is influenced by outiers.

In [8]:
store_data['CompetitionDistance'].fillna(store_data['CompetitionDistance'].median())

0        1270.0
1         570.0
2       14130.0
3         620.0
4       29910.0
         ...   
1110     1900.0
1111     1880.0
1112     9260.0
1113      870.0
1114     5350.0
Name: CompetitionDistance, Length: 1115, dtype: float64

**CompetitionOpenSinceMonth and CompetitionOpenSinceYear**	
- Since there is no information about these , I'm going to fill them with zero.

In [9]:
store_data.CompetitionOpenSinceMonth.fillna(0)
store_data.CompetitionOpenSinceYear.fillna(0)

0       2008.0
1       2007.0
2       2006.0
3       2009.0
4       2015.0
         ...  
1110    2014.0
1111    2006.0
1112       0.0
1113       0.0
1114       0.0
Name: CompetitionOpenSinceYear, Length: 1115, dtype: float64

Promo2SinceWeek, Promo2SinceYear and PromoInterval 
- In this case, the missing values comes from fields where Promo2 is 0, which means there are no ongoing promotions for those stores. As a result the related fields must also be 0. 

In [10]:
store_data.Promo2SinceWeek.fillna(0)
store_data.Promo2SinceYear.fillna(0)
store_data.PromoInterval.fillna(0)

0                      0
1        Jan,Apr,Jul,Oct
2        Jan,Apr,Jul,Oct
3                      0
4                      0
              ...       
1110     Jan,Apr,Jul,Oct
1111                   0
1112                   0
1113                   0
1114    Mar,Jun,Sept,Dec
Name: PromoInterval, Length: 1115, dtype: object

In [11]:
store_data['CompetitionOpenSinceMonth'] = store_data['CompetitionOpenSinceMonth'].fillna(0)
store_data['CompetitionOpenSinceYear'] = store_data['CompetitionOpenSinceYear'].fillna(0)

In [12]:
# Checking the null values again
store_data.isnull().sum()/store_data.shape[0] * 100 

Store                         0.000000
StoreType                     0.000000
Assortment                    0.000000
CompetitionDistance           0.269058
CompetitionOpenSinceMonth     0.000000
CompetitionOpenSinceYear      0.000000
Promo2                        0.000000
Promo2SinceWeek              48.789238
Promo2SinceYear              48.789238
PromoInterval                48.789238
dtype: float64

- No null values now.

In [13]:
# joining the train dataset to the store dataset using left join to ensure that all events are included
train_store_df = pd.merge(train_df, store_data, how='left', on='Store')
print("Train_Store dataset has {} Rows and {} variables".format(str(train_store_df.shape[0]),str(train_store_df.shape[1])))

Train_Store dataset has 844338 Rows and 19 variables


In [14]:
train_store_df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Weekend,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,Weekday,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,5,2015-07-31,6064,625,1,1,0,1,Weekday,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,Weekday,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,Weekday,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,5,2015-07-31,4822,559,1,1,0,1,Weekday,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


To accurately evaluate the performance various store types, it is important to consider the sales per customer.

In [15]:
train_store_df['SalesperCustomer']= train_store_df['Sales']/train_store_df['Customers']
train_store_df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Weekend,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,SalesperCustomer
0,1,5,2015-07-31,5263,555,1,1,0,1,Weekday,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,9.482883
1,2,5,2015-07-31,6064,625,1,1,0,1,Weekday,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",9.702400
2,3,5,2015-07-31,8314,821,1,1,0,1,Weekday,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",10.126675
3,4,5,2015-07-31,13995,1498,1,1,0,1,Weekday,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN,9.342457
4,5,5,2015-07-31,4822,559,1,1,0,1,Weekday,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN,8.626118
